<a href="https://colab.research.google.com/github/tejeshvaish/Recosense-Info-Solutions/blob/main/Cat_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [135]:
import nltk


In [136]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [137]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:
path = "/content/drive/MyDrive/Colab Notebooks/laptops.csv"
df_laptop = pd.read_csv(path,encoding = "ISO-8859-1")

In [140]:
df_laptop.head()

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [141]:
df_laptop = df_laptop[["Company",'Product']]

In [142]:
# df_laptop["cat"]= "Laptop"

In [143]:
df_laptop=df_laptop[:400]

In [144]:
path = "/content/drive/MyDrive/Colab Notebooks/mobile.csv"
df_mobile = pd.read_csv(path,encoding = "ISO-8859-1")

In [145]:
path = "/content/drive/MyDrive/Colab Notebooks/scrap_jakmal-komputer.csv"
df_acc = pd.read_csv(path,encoding = "ISO-8859-1")


In [146]:
df_mobile.shape

(2760, 7)

In [147]:
df_mobile=df_mobile[['name','item_brand_name']]

In [148]:
df_mobile.head()

,name,item_brand_name
0,Mione K1 ROM 3GB RAM 32GB 5.99 Inch display 4G...,mione
1,Porsche design Huawei Mate 20 RS Dual SIM - 512GB,huawei
2,Samsung Galaxy Fold Dual SIM - 512GB,samsung
3,Samsung Galaxy Fold Dual SIM - 512GB,samsung
4,Blackberry P'9981 Porsche Design - 8GB,blackberry


In [149]:
df_mobile=df_mobile[:400]

In [150]:
df_acc.head()

,seller_name,seller_location,seller_product,seller_product_price,seller_product_wishlistcount
0,Specialist Networking,DKI Jakarta,Inotrix SG411 Switch Poe Gigabit 4 Port +1 Por...,Rp 580.000,2
1,Redcom,DKI Jakarta,TRANSCEND Storejet 25H3 1TB Harddisk External ...,Rp 778.000,2
2,Indo Audio,DKI Jakarta,Taffware Bluetooth Audio USB External Soundcar...,Rp 159.200,96
3,zero promosi,Tangerang,FLASHDISK PLASTIK MODEL FLIP ON FDPL38,Rp 40.000,1
4,JOJO CompTech,DKI Jakarta,Corsair Carbide Series 110Q Quiet ATX Mid-Towe...,Rp 960.000,1


In [151]:
df_acc.shape

(60, 5)

In [152]:
df_acc = df_acc[['seller_product','seller_name']]

In [153]:
df_acc.head()

,seller_product,seller_name
0,Inotrix SG411 Switch Poe Gigabit 4 Port +1 Por...,Specialist Networking
1,TRANSCEND Storejet 25H3 1TB Harddisk External ...,Redcom
2,Taffware Bluetooth Audio USB External Soundcar...,Indo Audio
3,FLASHDISK PLASTIK MODEL FLIP ON FDPL38,zero promosi
4,Corsair Carbide Series 110Q Quiet ATX Mid-Towe...,JOJO CompTech


In [154]:
df_mobile.rename(columns={'name': 'Product', 'item_brand_name': 'Company'}, inplace=True)


In [155]:
df_laptop["cat"]="Laptop"
df_mobile["cat"]="Mobile"
df_acc["cat"]="Computer_Accesories"

In [162]:
# input_files = ['df_laptop.csv', 'df_mobile.csv', 'df_acc.csv']
df_laptop=df_laptop.append(df_mobile) 

In [163]:
df_acc.rename(columns={'seller_product': 'Product', 'seller_name': 'Company'}, inplace=True)


In [165]:
df=df_laptop.append(df_acc)

In [166]:
df.shape

(860, 3)

In [168]:
from sklearn.utils import shuffle
df = shuffle(df)

In [169]:
df

,Company,Product,cat
291,Asus,FX753VD-GC071T (i7-7700HQ/8GB/1TB/GeForce,Laptop
117,samsung,Samsung Galaxy S10 Plus Dual Sim - 1Tb,Mobile
347,Dell,Latitude 5590,Laptop
125,apple,Apple iPhone 11 Pro Max with FaceTime - 512GB,Mobile
30,samsung,Samsung Galaxy S20 Ultra Dual SIM - 128GB,Mobile
...,...,...,...
51,Acer,Aspire A515-51G,Laptop
200,Dell,Inspiron 7577,Laptop
124,Acer,Aspire 3,Laptop
300,mione,Mione K1 ROM 3GB RAM 32GB 5.99 Inch display 4G...,Mobile


In [176]:
test=df[770:]

In [178]:
train=df[:770]

In [179]:
test

,Company,Product,cat
119,Asus,VivoBook S15,Laptop
123,HP,Probook 470,Laptop
126,apple,Apple iPhone 11 Pro Max with FaceTime - 512GB,Mobile
305,Lenovo,IdeaPad 320-15IAP,Laptop
271,samsung,Samsung Galaxy S20 Ultra Dual SIM - 128GB,Mobile
...,...,...,...
51,Acer,Aspire A515-51G,Laptop
200,Dell,Inspiron 7577,Laptop
124,Acer,Aspire 3,Laptop
300,mione,Mione K1 ROM 3GB RAM 32GB 5.99 Inch display 4G...,Mobile


## **First I am training the model to find the category through product name**

In [180]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [182]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.cat.values)

In [184]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.Product.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [185]:
print (xtrain.shape)
print (xvalid.shape)

(693,)
(77,)


In [186]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [187]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

In [188]:
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 0.192 


In [189]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [190]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.165 


In [191]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.149 


In [192]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.018 


In [193]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [194]:
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.045 


SVM doesn't perform well on this dataset!


In [195]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 0.070 


In [196]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.065 


In [197]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.039 


In [198]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 0.040 


In [199]:
#Now applying grid Search!


In [200]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [201]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

In [202]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [203]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    2.4s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Best score: -0.069
Best parameters set:
	lr__C: 10
	lr__penalty: 'l2'
	svd__n_components: 120


Score is like SVM only





In [204]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.072
Best parameters set:
	nb__alpha: 0.01


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0127s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0326s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


### From the Above methods and differents Algo, a simple Naive Bayes on Counts gave the least log error of 0.018, its overfitting the data, since our data is too small, I am pretty much happy with it.

## **Now doin Analysis for Company name. **

As the number of companies is too huge in my dataset, so I am using  dependency parsing and extract the center word dependencies, and to print my Company name. It gives Correct Output maximum number of times!



In [228]:
import spacy
y_hat=[]
nlp = spacy.load("en_core_web_sm")
for index, row in train.iterrows():
    doc = nlp(row['Product'])
    # print(row['Product'])
    for chunk in doc.noun_chunks:
        y_hat.append(chunk.text)

y_hat=pd.DataFrame(y_hat)

In [229]:
# print ("logloss: %0.3f " % multiclass_logloss(train['Company'], y_hat))
print(y_hat.shape)
print(train.shape)

(1284, 1)
(770, 3)


In [231]:
import re
def clean_special_characters(txt):
  seps = [ " ",";",":",".", ",","*","#","@","|","/","-","_","?","%","(",")","^"]
  default_sep = seps[0]

  for sep in sep[1:]:
    txt = txt.replace(sep, default_sep)
    re.sub(' +',' ',txt)
    temp_list=[ i.strip() for i in txt.split(default_sep)]
    temp_list=[ i for i in temp_list if i]
    return " ".join(temp_list)
def clean_stopword(txt):
  temp_list = txt.split(" ")
  temp_list= [ i for i in temp_list if i not in stop_words]
  return " ".join(temp_list)

def data_cleaning(data, nameCol="Product",dropforign= True):
  data.dropna(subset=["Product"], inplace = True)
  data = data.rename_axis('Company').reset_index()
  data_clean = data.copy()
  data_clean['Product_clean']= data_clean[nameCol].apply(lambda x: x.lower())
  data_clean['Product_clean']= data_clean['Product_clean'].apply(clean_special_characters)
  data_clean['Product_clean']= data_clean['Product_clean'].apply(clean_stopword)




In [ ]:
def fuzz_similarity():